In [2]:
# When comparing two data sets using statistical tools, it is not
# sufficient to simply compare their average or median values. This is because
# such results can be skewed by randomness. For example, suppose we are
# calculating the median values of ten seeds. If one algorithm gets "lucky"
# and happens to use more above-average seeds, the estimated median will
# be skewed. Therefore, it is necessary to check the statistical significance of
# results. This is exactly what the indifferent entries are displaying. To determine
# statistical significance, the Kruskal-Wallis and Mann-Whitney U tests with 95% confidence
# intervals can be used.

# The Kruskal-Wallis H-test tests the null hypothesis that the population median of all
# of the groups are equal. It is a non-parametric version of ANOVA.
# The test works on 2 or more independent samples, which may have different sizes.
# Note that rejecting the null hypothesis does not indicate which of the groups differs.
# Post hoc comparisons between groups are required to determine which groups are different.

In [179]:
import numpy as np
from scipy import stats
import sys
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = [12, 8]
sys.path.append("..")
import rbf_functions

In [4]:
rbfs = [
    rbf_functions.original_rbf,
    rbf_functions.squared_exponential_rbf,
    rbf_functions.inverse_quadratic_rbf,
    rbf_functions.inverse_multiquadric_rbf,
    rbf_functions.exponential_rbf,
    rbf_functions.matern32_rbf,
    rbf_functions.matern52_rbf,
]

# Kruskal-Wallis HV local reference set

In [5]:
hv_global = pd.read_csv("./calculated_metrics/hv_refset_global.csv").iloc[0, -1]
averages = {}
for entry in rbfs:
    rbf = entry.__name__

    # normalize hv against reference set so reference set is 1
    df_hv = pd.read_csv(f"calculated_metrics/hv_{rbf}_global.csv")
    df_hv.hypervolume /= hv_global


for entry in rbfs:
    rbf = entry.__name__
    #     df_gd_data = pd.read_csv(f"gd_all/{rbf}_gd_all.csv")
    #     df_ei_data = pd.read_csv(f"ei_all/{rbf}_ei_all.csv")
    df_hv_data = pd.read_csv(f"hv/{rbf}_hv.csv")
    gddata[rbf] = {}
    eidata[rbf] = {}
    hvdata[rbf] = {}
    for i in varlist:  # add max hv value at 100.000 nfe to dict for each seed
        #         gddata[rbf][i] = df_gd_data[i].dropna().iloc[-1]
        #         eidata[rbf][i] = df_ei_data[i].dropna().iloc[-1]
        hvdata[rbf][i] = df_hv_data[i].dropna().iloc[-1]
df_gd = pd.DataFrame.from_dict(gddata, orient="index")
df_ei = pd.DataFrame.from_dict(eidata, orient="index")
df_hv = pd.DataFrame.from_dict(hvdata, orient="index")

In [187]:
df_kruskal = pd.DataFrame(columns=df_hv.index)
df_pval = pd.DataFrame(columns=df_hv.index)
for i, row in df_hv.iterrows():
    for j, row2 in df_hv.iterrows():
        krusval, pval = stats.kruskal(row, row2)
        df_kruskal.loc[i, j] = krusval
        df_pval.loc[i, j] = pval

In [171]:
df_kruskal

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,0.0,1.851429,13.165714,0.022857,14.285714,4.48,10.565714
squared_exponential_rbf,1.851429,0.0,3.862857,3.862857,11.571429,0.965714,4.165714
inverse_quadratic_rbf,13.165714,3.862857,0.0,14.285714,14.285714,2.062857,0.691429
inverse_multiquadric_rbf,0.022857,3.862857,14.285714,0.0,14.285714,8.691429,14.285714
exponential_rbf,14.285714,11.571429,14.285714,14.285714,0.0,7.0,4.48
matern32_rbf,4.48,0.965714,2.062857,8.691429,7.0,0.0,1.651429
matern52_rbf,10.565714,4.165714,0.691429,14.285714,4.48,1.651429,0.0


In [221]:
# df_pval
# df_k = df_kruskal.replace(0, np.nan) < 3.84146
x = 0
for i in df_k:
    x = x + df_k[i].values.sum()
x

12

In [190]:
df_pval.replace(0, np.nan) < 0.05

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,False,False,True,False,True,True,True
squared_exponential_rbf,False,False,True,True,True,False,True
inverse_quadratic_rbf,True,True,False,True,True,False,False
inverse_multiquadric_rbf,False,True,True,False,True,True,True
exponential_rbf,True,True,True,True,False,True,True
matern32_rbf,True,False,False,True,True,False,False
matern52_rbf,True,True,False,True,True,False,False


# Kruskal-Wallis on objectives

In [9]:
rbfd = {}
rbfn = []
for rbf in rbfs:
    name = rbf.__name__
    rbfn.append(name)
    rbfd[name] = pd.read_csv(f"refsets/{name}_refset.csv")

In [ ]:
# difference in variance (rank sum)
# null hypothesis > no difference between groups

# https://www.youtube.com/watch?v=l86wEhUzkY4

In [169]:
# a = rbfd[rbfn[0]]['hydropower']
# b = rbfd[rbfn[1]]['hydropower']
# c = rbfd[rbfn[2]]['hydropower']
# d = rbfd[rbfn[3]]['hydropower']
# e = rbfd[rbfn[4]]['hydropower']
# f = rbfd[rbfn[5]]['hydropower']
# stats.kruskal(a,b,c,d,e,f)

a = [14, 1, 3, 9, 5, 7]
b = [10, 6, 8, 18, 11, 12]
c = [17, 16, 2, 13, 15, 4]
stats.kruskal(a, b, c)
# 2.858 is not greater than 5.99, reject the null hypothesis > there is a difference

KruskalResult(statistic=2.853801169590639, pvalue=0.240051790590007)

# Hydropower

In [225]:
df_kruskal_hyd = pd.DataFrame(columns=rbfn)
df_pval_hyd = pd.DataFrame(columns=rbfn)
for i in rbfd:
    for j in rbfd:
        row1 = rbfd[i]["hydropower"]
        row2 = rbfd[j]["hydropower"]
        krusval, pval = stats.kruskal(row1, row2)
        df_kruskal_hyd.loc[i, j] = krusval
        df_pval_hyd.loc[i, j] = pval


df = df_kruskal_hyd.astype(float).round(2)
newcol = df.columns.to_list()
newcol = [x[:-4] for x in newcol]
df.columns = newcol
df.index = newcol

df_k = df_kruskal_hyd.replace(0, np.nan) < 3.84146
# x = 0
# for i in df_k:
#     x = x + df_k[i].values.sum()
# print(x)
df_k

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,False,True,False,False,False,False,False
squared_exponential_rbf,True,False,False,False,False,False,False
inverse_quadratic_rbf,False,False,False,False,False,False,False
inverse_multiquadric_rbf,False,False,False,False,True,False,False
exponential_rbf,False,False,False,True,False,False,False
matern32_rbf,False,False,False,False,False,False,False
matern52_rbf,False,False,False,False,False,False,False


In [196]:
df_pval_hyd.astype(float).round(2) < 0.05

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,False,False,True,True,True,True,True
squared_exponential_rbf,False,False,True,True,True,True,True
inverse_quadratic_rbf,True,True,False,True,True,True,True
inverse_multiquadric_rbf,True,True,True,False,False,True,True
exponential_rbf,True,True,True,False,False,True,True
matern32_rbf,True,True,True,True,True,False,True
matern52_rbf,True,True,True,True,True,True,False


# Atomic power plant

In [228]:
df_kruskal_atom = pd.DataFrame(columns=rbfn)
df_pval_atom = pd.DataFrame(columns=rbfn)
for i in rbfd:
    for j in rbfd:
        row1 = rbfd[i]["atomicpowerplant"]
        row2 = rbfd[j]["atomicpowerplant"]
        krusval, pval = stats.kruskal(row1, row2)
        df_kruskal_atom.loc[i, j] = krusval
        df_pval_atom.loc[i, j] = pval

df_k = df_kruskal_atom.replace(0, np.nan) < 3.84146
x = 0
for i in df_k:
    x = x + df_k[i].values.sum()
print(x)
df_k

12


,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,False,False,False,False,False,False,False
squared_exponential_rbf,False,False,False,False,False,True,True
inverse_quadratic_rbf,False,False,False,True,False,False,True
inverse_multiquadric_rbf,False,False,True,False,False,False,True
exponential_rbf,False,False,False,False,False,False,False
matern32_rbf,False,True,False,False,False,False,True
matern52_rbf,False,True,True,True,False,True,False


In [229]:
df_pval_atom < 0.05

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,False,True,True,True,True,True,True
squared_exponential_rbf,True,False,True,True,True,False,False
inverse_quadratic_rbf,True,True,False,False,True,True,False
inverse_multiquadric_rbf,True,True,False,False,True,True,False
exponential_rbf,True,True,True,True,False,True,True
matern32_rbf,True,False,True,True,True,False,False
matern52_rbf,True,False,False,False,True,False,False


# Baltimore

In [18]:
df_kruskal_balt = pd.DataFrame(columns=rbfn)
df_pval_balt = pd.DataFrame(columns=rbfn)
for i in rbfd:
    for j in rbfd:
        row1 = rbfd[i]["baltimore"]
        row2 = rbfd[j]["baltimore"]
        krusval, pval = stats.kruskal(row1, row2)
        df_kruskal_balt.loc[i, j] = krusval
        df_pval_balt.loc[i, j] = pval

In [231]:
df_k = df_kruskal_balt.replace(0, np.nan) < 3.84146
x = 0
for i in df_k:
    x = x + df_k[i].values.sum()
print(x)
df_k

8


,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,False,True,False,False,False,False,False
squared_exponential_rbf,True,False,False,False,False,False,False
inverse_quadratic_rbf,False,False,False,True,False,False,False
inverse_multiquadric_rbf,False,False,True,False,True,False,False
exponential_rbf,False,False,False,True,False,False,False
matern32_rbf,False,False,False,False,False,False,True
matern52_rbf,False,False,False,False,False,True,False


In [232]:
df_pval_balt < 0.05

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,False,False,True,True,True,True,True
squared_exponential_rbf,False,False,True,True,True,True,True
inverse_quadratic_rbf,True,True,False,False,True,True,True
inverse_multiquadric_rbf,True,True,False,False,False,True,True
exponential_rbf,True,True,True,False,False,True,True
matern32_rbf,True,True,True,True,True,False,False
matern52_rbf,True,True,True,True,True,False,False


# Chester

In [21]:
df_kruskal_ches = pd.DataFrame(columns=rbfn)
df_pval_ches = pd.DataFrame(columns=rbfn)
for i in rbfd:
    for j in rbfd:
        row1 = rbfd[i]["chester"]
        row2 = rbfd[j]["chester"]
        krusval, pval = stats.kruskal(row1, row2)
        df_kruskal_ches.loc[i, j] = krusval
        df_pval_ches.loc[i, j] = pval

In [233]:
df_k = df_kruskal_ches.replace(0, np.nan) < 3.84146
x = 0
for i in df_k:
    x = x + df_k[i].values.sum()
print(x)
df_k

6


,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,False,True,False,True,False,False,False
squared_exponential_rbf,True,False,False,True,False,False,False
inverse_quadratic_rbf,False,False,False,False,False,False,False
inverse_multiquadric_rbf,True,True,False,False,False,False,False
exponential_rbf,False,False,False,False,False,False,False
matern32_rbf,False,False,False,False,False,False,False
matern52_rbf,False,False,False,False,False,False,False


In [238]:
df_pval_ches < 0.05

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,False,False,True,False,True,True,True
squared_exponential_rbf,False,False,True,False,True,True,True
inverse_quadratic_rbf,True,True,False,True,True,True,True
inverse_multiquadric_rbf,False,False,True,False,True,True,True
exponential_rbf,True,True,True,True,False,True,True
matern32_rbf,True,True,True,True,True,False,True
matern52_rbf,True,True,True,True,True,True,False


# Environment

In [24]:
df_kruskal_env = pd.DataFrame(columns=rbfn)
df_pval_env = pd.DataFrame(columns=rbfn)
for i in rbfd:
    for j in rbfd:
        row1 = rbfd[i]["environment"]
        row2 = rbfd[j]["environment"]
        krusval, pval = stats.kruskal(row1, row2)
        df_kruskal_env.loc[i, j] = krusval
        df_pval_env.loc[i, j] = pval

In [234]:
df_k = df_kruskal_env.replace(0, np.nan) < 3.84146
x = 0
for i in df_k:
    x = x + df_k[i].values.sum()
print(x)
df_k

6


,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,False,True,False,False,False,False,False
squared_exponential_rbf,True,False,False,False,False,False,True
inverse_quadratic_rbf,False,False,False,False,False,False,False
inverse_multiquadric_rbf,False,False,False,False,False,False,False
exponential_rbf,False,False,False,False,False,True,False
matern32_rbf,False,False,False,False,True,False,False
matern52_rbf,False,True,False,False,False,False,False


In [235]:
df_pval_env < 0.05

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,False,False,True,True,True,True,True
squared_exponential_rbf,False,False,True,True,True,True,False
inverse_quadratic_rbf,True,True,False,True,True,True,True
inverse_multiquadric_rbf,True,True,True,False,True,True,True
exponential_rbf,True,True,True,True,False,False,True
matern32_rbf,True,True,True,True,False,False,True
matern52_rbf,True,False,True,True,True,True,False


# Recreation

In [25]:
df_kruskal_rec = pd.DataFrame(columns=rbfn)
df_pval_rec = pd.DataFrame(columns=rbfn)
for i in rbfd:
    for j in rbfd:
        row1 = rbfd[i]["recreation"]
        row2 = rbfd[j]["recreation"]
        krusval, pval = stats.kruskal(row1, row2)
        df_kruskal_rec.loc[i, j] = krusval
        df_pval_rec.loc[i, j] = pval

In [236]:
df_k = df_kruskal_rec.replace(0, np.nan) < 3.84146
x = 0
for i in df_k:
    x = x + df_k[i].values.sum()
print(x)
df_k

10


,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,False,True,False,True,False,False,False
squared_exponential_rbf,True,False,False,True,False,False,True
inverse_quadratic_rbf,False,False,False,False,False,False,False
inverse_multiquadric_rbf,True,True,False,False,False,False,False
exponential_rbf,False,False,False,False,False,False,False
matern32_rbf,False,False,False,False,False,False,True
matern52_rbf,False,True,False,False,False,True,False


In [237]:
df_pval_rec < 0.05

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,False,False,True,False,True,True,True
squared_exponential_rbf,False,False,True,False,True,True,False
inverse_quadratic_rbf,True,True,False,True,True,True,True
inverse_multiquadric_rbf,False,False,True,False,True,True,True
exponential_rbf,True,True,True,True,False,True,True
matern32_rbf,True,True,True,True,True,False,False
matern52_rbf,True,False,True,True,True,False,False


In [ ]:
# create index
df = df_kruskal_hyd.astype(float).round(2)
newcol = df.columns.to_list()
newcol = [x[:-4] for x in newcol]
df.columns = newcol
df.index = newcol

In [158]:
df = df_kruskal_rec.astype(float).round(2)
df2 = df_pval_rec.astype(float).round(2)
df.columns = newcol
df.index = newcol
df2.columns = newcol
df2.index = newcol
print("\\begin{table}[h]")
# print("\\parbox{.45\linewidth}{")
# print("\\centering")
print("\\begin{tabular}{c | " + " | ".join(["c"] * len(df.columns)) + "}")
# print(" & ".join([str(x) for x in newcol]) + " \\\\")
print(
    "& original & \makecell{squared\\\exponential} & \makecell{inverse\\\quadratic} & \makecell{inverse\\\multiquadric} & exponential & matern32 & matern52 \\\\"
)
print("\hline")
for i, row in df.iterrows():
    j = i.replace("_", " ")
    print(f"{j} & " + " & ".join([str(x) for x in row.values]) + " \\\\")
print("\\end{tabular}")
print("\\caption{Kruskal-Wallis test, recreation}")
print("\\end{table}")
print("")
# print("\\parbox{.45\linewidth}{")
# print("\\centering")
print("\\begin{table}[h]")
print("\\begin{tabular}{c | " + " | ".join(["c"] * len(df.columns)) + "}")
# print(" & " + " & ".join([str(x) for x in newcol]))
print(
    "& original & \makecell{squared\\\exponential} & \makecell{inverse\\\quadratic} & \makecell{inverse\\\multiquadric} & exponential & matern32 & matern52 \\\\"
)
print("\hline")
for i, row in df2.iterrows():
    j = i.replace("_", " ")
    print(f"{j} & " + " & ".join([str(x) for x in row.values]) + " \\\\")
print("\\end{tabular}")
print("\\caption{Kruskal-Wallis test, P value, recreation}")
# print("}")
print("\\end{table}")

\begin{table}[h]
\begin{tabular}{c | c | c | c | c | c | c | c}
& original & \makecell{squared\\exponential} & \makecell{inverse\\quadratic} & \makecell{inverse\\multiquadric} & exponential & matern32 & matern52 \\
\hline
original & 0.0 & 2.41 & 205.16 & 0.25 & 118.36 & 13.05 & 8.5 \\
squared exponential & 2.41 & 0.0 & 159.51 & 3.54 & 86.69 & 4.24 & 2.35 \\
inverse quadratic & 205.16 & 159.51 & 0.0 & 169.15 & 6.73 & 108.98 & 88.51 \\
inverse multiquadric & 0.25 & 3.54 & 169.15 & 0.0 & 102.44 & 13.58 & 9.51 \\
exponential & 118.36 & 86.69 & 6.73 & 102.44 & 0.0 & 52.65 & 45.29 \\
matern32 & 13.05 & 4.24 & 108.98 & 13.58 & 52.65 & 0.0 & 0.09 \\
matern52 & 8.5 & 2.35 & 88.51 & 9.51 & 45.29 & 0.09 & 0.0 \\
\end{tabular}
\caption{Kruskal-Wallis test, recreation}
\end{table}

\begin{table}[h]
\begin{tabular}{c | c | c | c | c | c | c | c}
& original & \makecell{squared\\exponential} & \makecell{inverse\\quadratic} & \makecell{inverse\\multiquadric} & exponential & matern32 & matern52 \\
\hlin